# 1. DATA LOADING AND PREPARATION

In this first part we will find the following:

- Libraries
- Loading of data
- Feature description

In [1]:
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
import lazypredict
from lazypredict.Supervised import LazyClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

In [3]:
#Load Data
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [4]:
print(train.shape)
train.head()

(4392, 17)


,customer_id,customer_age,education_level,marital_status,income_category,number_products_customer,weeks_tenure,contacts_last_12mths,credit_limit,card_class,inactive_months_last_12mths,total_revolving_balance,count_transactions,transactions_amount,change_transaction_amt_last_3mths,change_transaction_count_last_3mths,churn
0,4278,NaN,phd,NaN,30k€-50k€,4,216,1,16541.00,red plus,2,1428,82,4352,0.85,0.91,0
1,7300,45.00,graduate,single,<30k€,5,115,3,1438.30,red,3,576,66,5098,0.77,0.83,0
2,6857,NaN,uneducated,married,30k€-50k€,6,172,2,2293.00,red,1,1460,74,4436,0.66,0.54,0
3,2508,39.00,graduate,married,70k€-110k€,5,124,2,17523.00,red,2,0,54,1974,0.42,0.69,1
4,7859,40.00,graduate,single,<30k€,2,107,2,4391.00,red,2,1933,36,1849,0.58,0.39,1


In [5]:
print(test.shape)
test.tail()

(1883, 16)


,customer_id,customer_age,education_level,marital_status,income_category,number_products_customer,weeks_tenure,contacts_last_12mths,credit_limit,card_class,inactive_months_last_12mths,total_revolving_balance,count_transactions,transactions_amount,change_transaction_amt_last_3mths,change_transaction_count_last_3mths
1878,5704,50.00,phd,married,<30k€,3,163,4,3547.00,red,4,2517,45,2715,0.81,0.73
1879,6371,49.00,NaN,single,<30k€,2,153,1,2105.00,red,2,1072,81,4833,0.51,1.08
1880,3073,41.00,NaN,married,30k€-50k€,5,122,2,2166.00,red,2,1465,82,5097,0.86,0.67
1881,1912,52.00,graduate,married,<30k€,5,200,0,5097.00,red,1,1152,45,2052,0.94,0.61
1882,3196,45.00,uneducated,married,70k€-110k€,6,153,2,22913.00,red,2,1528,35,1394,0.41,0.52


### Feature description

- **customer_id:** id of the client. It is a unique value per client.
- **customer_age:** age of the client.
- **education_level:** level of education: graduate, high school, uneducated, college, post-graduate, phd.
- **marital_status:** married, single and divorced.
- **income_category:** annual income level by category: <30k€, 30k€-50k€, 50k€-70k€, 70k€-110k€, +110k€. 
- **number_products_customer:** number of products that the customer has contracted with the bank.
- **weeks_tenure:** number of weeks since the customer first signed up for a product with the bank.
- **contacts_last_12mths:** interactions between the customer and the bank on both sides during the last year.
- **credit_limit:** limit on the card.
- **card_class:** each type has diferents advantages. categories: red, red plus, red unlimited and premium.
- **inactive_months_last_12mths:** number of months in the last year in which the customer has not used the credit card.
- **total_revolving_balance:** amount of debt that has not been repaid in one cycle, which is carried over to the next, with a consequent increase in interest.
- **count_transactions:** number of transactions made with the card in the last year.
- **transactions_amount:** total of money involved in last year's transactions. 
- **change_transaction_amt_last_3mths:** relative difference in the amount of money spent in the last 3 months. A value greater than 1 indicates an increase in spending.
- **change_transaction_count_last_3mthn:**  relative difference in the number of transactions in the last 3 months. A value greater than 1 indicates an increase in the number of transactions.
- **churn:** "0" if the customer is still with the bank, "1" if the customer has cancelled his credit card.


*Notes:* 

- *The loading of the data has been successful.*
- *The training dataset has 4392 observations and 17 variables, the test dataset has 1883 observations and 16 variables.*

# 2. Data cleaning and preprocessing

The first step of this second part will be to join the training and test data, to each dataset a variable 'type' will be added, which will indicate to which dataset it initially belongs (train/test). 

The target variable is also specified, this will not be used until the final step of this part.

At this point we will mainly work with the dataframe 'train_test_df'.

In [6]:
y = train['churn'] #Target variable


train['type']='train'
train.drop('churn', axis=1, inplace = True)
print(train.shape)


test['type']='test'
print(test.shape)


train_test_df=pd.concat([train,test])
train_test_df.shape

(4392, 17)
(1883, 17)


(6275, 17)

In [7]:
train_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6275 entries, 0 to 1882
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          6275 non-null   int64  
 1   customer_age                         5651 non-null   float64
 2   education_level                      5339 non-null   object 
 3   marital_status                       5815 non-null   object 
 4   income_category                      5572 non-null   object 
 5   number_products_customer             6275 non-null   int64  
 6   weeks_tenure                         6275 non-null   int64  
 7   contacts_last_12mths                 6275 non-null   int64  
 8   credit_limit                         6275 non-null   float64
 9   card_class                           6275 non-null   object 
 10  inactive_months_last_12mths          6275 non-null   int64  
 11  total_revolving_balance       

After obtaining the general information from the data and with the previous description of the variables, the following steps can be drawn.

- The variable 'customer_id' is not relevant for the model.
- There are four categorical variables: 'education_level', 'marital_status', 'income_category' and 'card_class'. The first and third are ordinal categorical variables. \
From the last one it can be intuited that there is a category level, however the description specifies that they simply have different advantage.
- The numeric variables; of type 'int' will be: 'customer_age', 'number_products_customer', 'weeks_tenure', 'contacts_last_12mths', 'inactive_months_last_12mths' and 'count_transactions'; of type 'float' shall be those referring to currency or ratios: 'credit_limit', 'total_revolving_balance', 'transactions_amount', 'change_transaction_amt_last_3mths' and 'change_transaction_count_last_3mths'.\
Therefore, the variables that will need to be corrected are: 'customer_age', 'total_revolving_balance' and ' transactions_amount'.

In [8]:
# Drop duplicates
train_test_df.drop_duplicates() #if we have any customer id duplicated, this will be deleted.
print(train_test_df.shape)

(6275, 17)


In [9]:
#Missing values 
total_missing = train_test_df.isna().sum().sort_values(ascending=False)
print(total_missing)

# how many total missing values do we have?
total_cells = np.product(train_test_df.shape)

# percent of data that is missing
percent_missing = (total_missing.sum()/total_cells) * 100
print("Missing value Percentage:", round(percent_missing, 2), "%")

education_level                        936
income_category                        703
customer_age                           624
marital_status                         460
customer_id                              0
total_revolving_balance                  0
change_transaction_count_last_3mths      0
change_transaction_amt_last_3mths        0
transactions_amount                      0
count_transactions                       0
credit_limit                             0
inactive_months_last_12mths              0
card_class                               0
contacts_last_12mths                     0
weeks_tenure                             0
number_products_customer                 0
type                                     0
dtype: int64
Missing value Percentage: 2.55 %


In [10]:
#Missing data by feature

print("education_level:", round((total_missing[0]/train_test_df.shape[0])*100,2), "%")
print("income_category:", round((total_missing[1]/train_test_df.shape[0])*100,2), "%")
print("customer_age:", round((total_missing[2]/train_test_df.shape[0])*100,2), "%")
print("marital_status:", round((total_missing[3]/train_test_df.shape[0])*100,2), "%")

education_level: 14.92 %
income_category: 11.2 %
customer_age: 9.94 %
marital_status: 7.33 %


We can see that we have four variables containing 'Nan' values: the first one, 'education_level', is missing about 15% of the data, then 'income_category', which is missing about 11%, followed by 'customer_age', with about 10% of missing data and finally, 'marital_status', with about 7% of missing data. \
We have these missing data in 1 numerical variable and 3 categorical variables, two of them ordinal and one no ordinal variable.

### Imputing Missing Values

There are different methodologies, the most notable being 'SimpleImputer', 'KNNImputer' or 'IterativeImputer'. In this project we are going to use the first one mentioned, for the numerical variable, it will be imputed by the median and for the categorical variables by the mode. 

In [11]:
#Replacing Nan per mode
imputer_cols=['education_level', 'income_category', 'marital_status']
imputer = SimpleImputer(strategy='most_frequent')
#imputer.fit(train[imputer_cols])
train_test_df[imputer_cols] = imputer.fit_transform(train_test_df[imputer_cols])

In [12]:
imp=SimpleImputer(strategy='median')
#imp.fit(train.customer_age)
train_test_df['customer_age']=imp.fit_transform(train_test_df['customer_age'].to_numpy().reshape(-1,1))

In [13]:
train_test_df.head(10)

,customer_id,customer_age,education_level,marital_status,income_category,number_products_customer,weeks_tenure,contacts_last_12mths,credit_limit,card_class,inactive_months_last_12mths,total_revolving_balance,count_transactions,transactions_amount,change_transaction_amt_last_3mths,change_transaction_count_last_3mths,type
0,4278,46.00,phd,married,30k€-50k€,4,216,1,16541.00,red plus,2,1428,82,4352,0.85,0.91,train
1,7300,45.00,graduate,single,<30k€,5,115,3,1438.30,red,3,576,66,5098,0.77,0.83,train
2,6857,46.00,uneducated,married,30k€-50k€,6,172,2,2293.00,red,1,1460,74,4436,0.66,0.54,train
3,2508,39.00,graduate,married,70k€-110k€,5,124,2,17523.00,red,2,0,54,1974,0.42,0.69,train
4,7859,40.00,graduate,single,<30k€,2,107,2,4391.00,red,2,1933,36,1849,0.58,0.39,train
5,6965,54.00,uneducated,divorced,<30k€,6,191,1,2201.00,red,6,1420,70,4256,0.76,0.59,train
6,1372,37.00,high school,married,70k€-110k€,5,123,3,11229.00,red,3,1054,36,1898,0.55,1.57,train
7,8424,50.00,graduate,married,70k€-110k€,5,146,4,7290.00,red,1,2151,29,1312,0.58,0.53,train
8,8582,39.00,graduate,married,70k€-110k€,2,129,3,28687.00,red,1,1607,90,7523,0.75,0.64,train
9,8898,48.00,graduate,married,<30k€,3,147,4,1637.00,red,1,774,78,4120,1.13,1.00,train


### Encoding Categorical Features

In this section two types of methods will be used: 'Ordinal Encoder' for the categorical variables that have levels and 'One-Hot Encoder' for the rest.

In [14]:
# Encoding categorical variables - Ordinal Encoder
enc = OrdinalEncoder(categories=[['uneducated', 'high school', 'college', 'graduate', 'post-graduate', 'phd']])
enc.fit(train_test_df[['education_level']])
train_test_df['educ-enc']=enc.transform(train_test_df[['education_level']])
train_test_df.head()

,customer_id,customer_age,education_level,marital_status,income_category,number_products_customer,weeks_tenure,contacts_last_12mths,credit_limit,card_class,inactive_months_last_12mths,total_revolving_balance,count_transactions,transactions_amount,change_transaction_amt_last_3mths,change_transaction_count_last_3mths,type,educ-enc
0,4278,46.00,phd,married,30k€-50k€,4,216,1,16541.00,red plus,2,1428,82,4352,0.85,0.91,train,5.00
1,7300,45.00,graduate,single,<30k€,5,115,3,1438.30,red,3,576,66,5098,0.77,0.83,train,3.00
2,6857,46.00,uneducated,married,30k€-50k€,6,172,2,2293.00,red,1,1460,74,4436,0.66,0.54,train,0.00
3,2508,39.00,graduate,married,70k€-110k€,5,124,2,17523.00,red,2,0,54,1974,0.42,0.69,train,3.00
4,7859,40.00,graduate,single,<30k€,2,107,2,4391.00,red,2,1933,36,1849,0.58,0.39,train,3.00


In [15]:
enc_inc=OrdinalEncoder(categories=[['<30k€','30k€-50k€','50k€-70k€','70k€-110k€','+110k€']])
enc_inc.fit(train_test_df[['income_category']])
train_test_df['inc-enc']=enc_inc.transform(train_test_df[['income_category']])
train_test_df.head()

,customer_id,customer_age,education_level,marital_status,income_category,number_products_customer,weeks_tenure,contacts_last_12mths,credit_limit,card_class,inactive_months_last_12mths,total_revolving_balance,count_transactions,transactions_amount,change_transaction_amt_last_3mths,change_transaction_count_last_3mths,type,educ-enc,inc-enc
0,4278,46.00,phd,married,30k€-50k€,4,216,1,16541.00,red plus,2,1428,82,4352,0.85,0.91,train,5.00,1.00
1,7300,45.00,graduate,single,<30k€,5,115,3,1438.30,red,3,576,66,5098,0.77,0.83,train,3.00,0.00
2,6857,46.00,uneducated,married,30k€-50k€,6,172,2,2293.00,red,1,1460,74,4436,0.66,0.54,train,0.00,1.00
3,2508,39.00,graduate,married,70k€-110k€,5,124,2,17523.00,red,2,0,54,1974,0.42,0.69,train,3.00,3.00
4,7859,40.00,graduate,single,<30k€,2,107,2,4391.00,red,2,1933,36,1849,0.58,0.39,train,3.00,0.00


In [16]:
# Encoding categorical variables -- One-hot
cat_onehot_feat=['card_class', 'marital_status']
dummies = pd.get_dummies(train_test_df[cat_onehot_feat], drop_first=True)
print(dummies.head())

train_test_df = pd.concat([train_test_df, dummies], axis=1)
train_test_df.head()

   card_class_red  card_class_red plus  card_class_red unlimited  \
0               0                    1                         0   
1               1                    0                         0   
2               1                    0                         0   
3               1                    0                         0   
4               1                    0                         0   

   marital_status_married  marital_status_single  
0                       1                      0  
1                       0                      1  
2                       1                      0  
3                       1                      0  
4                       0                      1  


,customer_id,customer_age,education_level,marital_status,income_category,number_products_customer,weeks_tenure,contacts_last_12mths,credit_limit,card_class,...,change_transaction_amt_last_3mths,change_transaction_count_last_3mths,type,educ-enc,inc-enc,card_class_red,card_class_red plus,card_class_red unlimited,marital_status_married,marital_status_single
0,4278,46.00,phd,married,30k€-50k€,4,216,1,16541.00,red plus,...,0.85,0.91,train,5.00,1.00,0,1,0,1,0
1,7300,45.00,graduate,single,<30k€,5,115,3,1438.30,red,...,0.77,0.83,train,3.00,0.00,1,0,0,0,1
2,6857,46.00,uneducated,married,30k€-50k€,6,172,2,2293.00,red,...,0.66,0.54,train,0.00,1.00,1,0,0,1,0
3,2508,39.00,graduate,married,70k€-110k€,5,124,2,17523.00,red,...,0.42,0.69,train,3.00,3.00,1,0,0,1,0
4,7859,40.00,graduate,single,<30k€,2,107,2,4391.00,red,...,0.58,0.39,train,3.00,0.00,1,0,0,0,1


In [17]:
# Variable type transformation
train_test_df = train_test_df.astype({'customer_age': int, 
                                      'total_revolving_balance': float, 
                                      'transactions_amount': float})


We proceed to eliminate the variables that have been transformed and additionally the 'customer_id' variable; we also separate the dataset again as initially had, in training and test data using the 'type' variable, once separated this will also be eliminated.

In [18]:
#Drop unnecessary variables 
train_test_df=train_test_df.drop(['customer_id', 'education_level', 'marital_status','income_category', 'card_class'], axis=1)
train_test_df.head()

,customer_age,number_products_customer,weeks_tenure,contacts_last_12mths,credit_limit,inactive_months_last_12mths,total_revolving_balance,count_transactions,transactions_amount,change_transaction_amt_last_3mths,change_transaction_count_last_3mths,type,educ-enc,inc-enc,card_class_red,card_class_red plus,card_class_red unlimited,marital_status_married,marital_status_single
0,46,4,216,1,16541.00,2,1428.00,82,4352.00,0.85,0.91,train,5.00,1.00,0,1,0,1,0
1,45,5,115,3,1438.30,3,576.00,66,5098.00,0.77,0.83,train,3.00,0.00,1,0,0,0,1
2,46,6,172,2,2293.00,1,1460.00,74,4436.00,0.66,0.54,train,0.00,1.00,1,0,0,1,0
3,39,5,124,2,17523.00,2,0.00,54,1974.00,0.42,0.69,train,3.00,3.00,1,0,0,1,0
4,40,2,107,2,4391.00,2,1933.00,36,1849.00,0.58,0.39,train,3.00,0.00,1,0,0,0,1


In [19]:
X = train_test_df[train_test_df.type=='train']
X=X.drop('type', axis=1)
print(X.shape)
X_Test=train_test_df[train_test_df.type=='test']
X_Test=X_Test.drop('type', axis=1)
print(X_Test.shape)

(4392, 18)
(1883, 18)


In [20]:
X.head()

,customer_age,number_products_customer,weeks_tenure,contacts_last_12mths,credit_limit,inactive_months_last_12mths,total_revolving_balance,count_transactions,transactions_amount,change_transaction_amt_last_3mths,change_transaction_count_last_3mths,educ-enc,inc-enc,card_class_red,card_class_red plus,card_class_red unlimited,marital_status_married,marital_status_single
0,46,4,216,1,16541.00,2,1428.00,82,4352.00,0.85,0.91,5.00,1.00,0,1,0,1,0
1,45,5,115,3,1438.30,3,576.00,66,5098.00,0.77,0.83,3.00,0.00,1,0,0,0,1
2,46,6,172,2,2293.00,1,1460.00,74,4436.00,0.66,0.54,0.00,1.00,1,0,0,1,0
3,39,5,124,2,17523.00,2,0.00,54,1974.00,0.42,0.69,3.00,3.00,1,0,0,1,0
4,40,2,107,2,4391.00,2,1933.00,36,1849.00,0.58,0.39,3.00,0.00,1,0,0,0,1


In [21]:
# Subdivision of the training data
X_train , X_test , y_train , y_test = train_test_split(X , 
                                                       y, 
                                                       random_state = 23 ,
                                                       test_size =0.30)

*Notes:*

- *X_test ≠ X_Test*. The first one is a subset of the training data. The second one bellow to the original test data.
- *In this section, duplicate observations have been eliminated, 'Nan' values have been observed and imputed, and categorical variables have been coded.*

# 3. Modelling Data

In the following, the modelling section will be divided into three parts, the first and the second part will use the training fata and the third part will include the original test data.


### 3.1. LazyPredict

We will use the package 'Lazypredict - LazyClassifier', this package solves classification problems with basic models and helps to understand which ones work better without tuned parameter.

In [22]:
#Modelamos
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None,
                     predictions=False,
                     random_state=23,
                     classifiers='all')

models, predictions = clf.fit(X_train , X_test , y_train , y_test)

100%|██████████| 29/29 [00:06<00:00,  4.39it/s]


In [23]:
#Top10 results
models[:10]

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.96,0.95,0.95,0.96,0.25
XGBClassifier,0.95,0.94,0.94,0.95,0.30
RandomForestClassifier,0.96,0.93,0.93,0.95,0.62
BaggingClassifier,0.94,0.92,0.92,0.94,0.17
AdaBoostClassifier,0.94,0.92,0.92,0.94,0.32
DecisionTreeClassifier,0.92,0.90,0.90,0.92,0.07
ExtraTreesClassifier,0.93,0.89,0.89,0.93,0.42
SVC,0.91,0.86,0.86,0.91,0.34
LinearDiscriminantAnalysis,0.88,0.83,0.83,0.88,0.05


### 3.2 LGBM Classifier + GridSearchCV

With the first result of the LazyClassifier, 'LGBM Classifier' and 'GridSearchCV', we will try to improve on the result obtained previously.

In [24]:
#define grid parameters
lgbm_params = {
    "n_estimators":[100, 200, 300],
    "learning_rate":[0.01, 0.05, 0.1, 0.3],
    "num_leaves": [20, 50, 80, 100]
}

#define classifier
lgbmc = LGBMClassifier(seed=23,
                       objective='binary',
                       is_unbalance=True)

#GridSearch
grid_search = GridSearchCV(
    estimator=lgbmc,
    param_grid=lgbm_params,
    n_jobs=-1,
    cv=3,
    scoring='accuracy',
    error_score=0)

grid_result = grid_search.fit(X_train, y_train)

#set best parameters
model = lgbmc.set_params(**grid_result.best_params_)

#fit model
model.fit(X_train, y_train)

#prediction
y_pred = model.predict(X_test)

#Results
print(lgbmc.score(X_test, y_test))

0.9658573596358119


### 4.3 Final Prediction

In [25]:
#Submission
submission = pd.read_csv("/content/sample_submission.csv")

submission['churn'] = model.predict(X_Test)
submission.head()

,customer_id,churn
0,4527,0
1,4166,0
2,4409,0
3,6376,0
4,6367,0


In [26]:
#Save submission 
submission.to_csv("submission.csv", index=False)